In [1]:
try:
    import google.colab
    google.colab.drive.mount('/content/drive/')
    %cd ../content/drive/Othercomputers/Laptop/Flood_Detection
except:
    pass

In [2]:
import sys; sys.path.append('../../')
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from mlpath import mlquest as mlq
from DataPreparation.DataPreparation import read_data
from FeatureExtraction.Hist.Hist import apply_hist
from sklearn.metrics import classification_report
mlq.start_quest("HOG-LR", table_dest="../../", log_defs=True)
sys.path.append('../')
from FeatureExtraction.SimpleFeatures.SimpleFeatures import *

### Reading and Preprocessing

In [3]:
x_train_d, x_val_d, y_train_d, y_val_d = mlq.l(read_data)(gray=True,saved=True)

x_train shape:  (737, 256, 256)
x_val shape:  (185, 256, 256)


### Feature Extraction: HOG Features

In [4]:
x_train_f = mlq.l(get_hog)(x_train_d)
x_val_f= mlq.l(get_hog)(x_val_d)

### Dimensionality Reduction: PCA

In [5]:
pca,x_train_f=mlq.l(apply_pca)(x_train_f,n_components=512,pca_obj=None)
x_val_f=mlq.l(apply_pca)(x_val_f,n_components=512,pca_obj=pca)

### Initiating the Model

In [6]:
model = mlq.l(LogisticRegression)(penalty='l2', C=7)

### Training the Model

In [7]:
model.fit(x_train_f, y_train_d)

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=7)

### Validating the Model

In [8]:
F1 = metrics.f1_score(y_val_d, model.predict(x_val_f), average='macro').item()
Accuracy = metrics.accuracy_score(y_val_d, model.predict(x_val_f)).item()
mlq.log_metrics(Accuracy, F1)
report = classification_report(y_val_d, model.predict(x_val_f))
print(report)

              precision    recall  f1-score   support

           0       0.78      0.87      0.83        87
           1       0.88      0.79      0.83        98

    accuracy                           0.83       185
   macro avg       0.83      0.83      0.83       185
weighted avg       0.83      0.83      0.83       185



### Tracking

In [9]:
mlq.end_quest()
mlq.show_logs(quest_name='HOG-LR', table_dest='../../', last_k=5)